# Feature Extraction

## Reading the input

In [1]:
import numpy as np
import pyedflib
from matplotlib import pyplot as plt
from nitime import utils
from nitime import algorithms as alg
from nitime.timeseries import TimeSeries
from nitime.viz import plot_tseries
import csv
import pywt
import scipy.stats as sp
from spectrum import *
from os import listdir
from os.path import isfile, join
import mne.io

C:\Users\mirab\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:107: MatplotlibDeprecationWarning: The mpl_toolkits.axes_grid module was deprecated in version 2.1. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist provies the same functionality instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


#### Names of all features

In [2]:
names = ['Activity','Mobility','Complexity','Kurtosis','2nd Difference Mean','2nd Difference Max','Coeffiecient of Variation','Skewness','1st Difference Mean','1st Difference Max',
          'Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Energy',
          'Wavelet Approximate Entropy','Wavelet Detailed Entropy','Variance','Mean of Vertex to Vertex Slope','FFT Delta MaxPower','FFT Theta MaxPower','FFT Alpha MaxPower','FFT Beta MaxPower',
          'Autro Regressive Mode Order 3 Coefficients for each channel ->']

# Hjorth Parameters    

In [3]:
def hjorth(input):                                             # function for hjorth 
    realinput = input
    hjorth_activity = np.zeros(len(realinput))
    hjorth_mobility = np.zeros(len(realinput))
    hjorth_diffmobility = np.zeros(len(realinput))
    hjorth_complexity = np.zeros(len(realinput))
    diff_input = np.diff(realinput)
    diff_diffinput = np.diff(diff_input)
    k = 0
    for j in realinput:
        hjorth_activity[k] = np.var(j)
        hjorth_mobility[k] = np.sqrt(np.var(diff_input[k])/hjorth_activity[k])
        hjorth_diffmobility[k] = np.sqrt(np.var(diff_diffinput[k])/np.var(diff_input[k]))
        hjorth_complexity[k] = hjorth_diffmobility[k]/hjorth_mobility[k]
        k = k+1
    return np.sum(hjorth_activity)/14, np.sum(hjorth_mobility)/14, np.sum(hjorth_complexity)/14                       #returning hjorth activity, hjorth mobility , hjorth complexity

# Kurtosis , 2nd Diff Mean, 2nd Diff Max

In [4]:
def my_kurtosis(a):
    b = a # Extracting the data from the 14 channels
    output = np.zeros(len(b)) # Initializing the output array with zeros (length = 14)
    k = 0; # For counting the current row no.
    for i in b:
        mean_i = np.mean(i) # Saving the mean of array i
        std_i = np.std(i) # Saving the standard deviation of array i
        t = 0.0
        for j in i:
            t += (pow((j-mean_i)/std_i,4)-3)
        kurtosis_i = t/len(i) # Formula: (1/N)*(summation(x_i-mean)/standard_deviation)^4-3
        output[k] = kurtosis_i # Saving the kurtosis in the array created
        k +=1 # Updating the current row no.
    return np.sum(output)/14

##----------------------------------------- End Kurtosis Function ----------------------------##


##------------------------------------- Begin 2ndDiffMean(Absolute difference) Function ------##
##-------------------------- [ Input: 2D array (row: Channels, column: Data)] --------------- ##
##-------------------  -- [ Output: 1D array (2ndDiffMean values for each channel)] ----------##

def secDiffMean(a):
    b = a # Extracting the data of the 14 channels
    output = np.zeros(len(b)) # Initializing the output array with zeros (length = 14)
    temp1 = np.zeros(len(b[0])-1) # To store the 1st Diffs
    k = 0; # For counting the current row no.
    for i in b:
        t = 0.0
        for j in range(len(i)-1):
            temp1[j] = abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
        for j in range(len(i)-2):
            t += abs(temp1[j+1]-temp1[j]) # Summing the 2nd Diffs
        output[k] = t/(len(i)-2) # Calculating the mean of the 2nd Diffs
        k +=1 # Updating the current row no.
    return np.sum(output)/14

##------------------------------------- End 2ndDiffMean Function----- -------------------------##


##------------------------------------- Begin 2ndDiffMax Function(Absolute difference) --------##
##-------------------------- [ Input: 2D array (row: Channels, column: Data)] -----------------##
##--------------------- [ Output: 1D array (2ndDiffMax values for each channel)] --------------##

def secDiffMax(a):
    b = a # Extracting the data from the 14 channels
    output = np.zeros(len(b)) # Initializing the output array with zeros (length = 14)
    temp1 = np.zeros(len(b[0])-1) # To store the 1st Diffs
    k = 0; # For counting the current row no.
    t = 0.0
    for i in b:
        for j in range(len(i)-1):
            temp1[j] = abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
            
        t = temp1[1] - temp1[0]
        for j in range(len(i)-2):
            if abs(temp1[j+1]-temp1[j]) > t :
            	t = temp1[j+1]-temp1[j] # Comparing current Diff with the last updated Diff Max

        output[k] = t # Storing the 2nd Diff Max for channel k
        k +=1 # Updating the current row no.
    return np.sum(output)/14



def wrapper1(a):
    kurtosis =  my_kurtosis(a)
    sec_diff_mean = secDiffMean(a)
    sec_diff_max  = secDiffMax(a)
    return kurtosis,sec_diff_mean,sec_diff_max

# Coefficient of Varaition

In [5]:
def coeff_var(a):
    b = a #Extracting the data from the 14 channels
    output = np.zeros(len(b)) #Initializing the output array with zeros
    k = 0; #For counting the current row no.
    for i in b:
        mean_i = np.mean(i) #Saving the mean of array i
        std_i = np.std(i) #Saving the standard deviation of array i
        output[k] = std_i/mean_i #computing coefficient of variation
        k=k+1
    return np.sum(output)/14

# Skewness , 1st Difference Mean, 1st Difference Max

In [6]:
def skewness(arr):
    data = arr 
    skew_array = np.zeros(len(data)) #Initialinling the array as all 0s
    index = 0; #current cell position in the output array
   
    for i in data:
        skew_array[index]=sp.stats.skew(i,axis=0,bias=True)
        index+=1 #updating the cell position
    return np.sum(skew_array)/14


def first_diff_mean(arr):
    data = arr 
    diff_mean_array = np.zeros(len(data)) #Initialinling the array as all 0s
    index = 0; #current cell position in the output array
   
    for i in data:
        sum=0.0#initializing the sum at the start of each iteration
        for j in range(len(i)-1):
            sum += abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
           
        diff_mean_array[index]=sum/(len(i)-1)
        index+=1 #updating the cell position
    return np.sum(diff_mean_array)/14


def first_diff_max(arr):
    data = arr 
    diff_max_array = np.zeros(len(data)) #Initialinling the array as all 0s
    first_diff = np.zeros(len(data[0])-1)#Initialinling the array as all 0s 
    index = 0; #current cell position in the output array
   
    for i in data:
        max=0.0#initializing at the start of each iteration
        for j in range(len(i)-1):
            first_diff[j] = abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
            if first_diff[j]>max: 
                max=first_diff[j] # finding the maximum of the first differences
        diff_max_array[index]=max
        index+=1 #updating the cell position
    return np.sum(diff_max_array)/14


def wrapper2(arr):
    skew   = skewness(arr)
    fdmean = first_diff_mean(arr)
    fdmax  = first_diff_max(arr)
    return skew,fdmean,fdmax

# Wavelet transform features

In [31]:
def wavelet_features(epoch):
    cA_values = []
    cD_values = []
    cA_mean = []
    cA_std = []
    cA_Energy =[]
    cD_mean = []
    cD_std = []
    cD_Energy = []
    Entropy_D = []
    Entropy_A = []
    for i in range(14):
        cA,cD=pywt.dwt(epoch[i,:],'coif1')
        cA_values.append(cA)
        cD_values.append(cD)		#calculating the coefficients of wavelet transform.
    for x in range(14):   
        cA_mean.append(np.mean(cA_values[x]))
        cA_std.append(np.std(cA_values[x]))
        cA_Energy.append(np.sum(np.square(cA_values[x])))
        cD_mean.append(np.mean(cD_values[x]))		# mean and standard deviation values of coefficents of each channel is stored .
        cD_std.append(np.std(cD_values[x]))
        cD_Energy.append(np.sum(np.square(cD_values[x])))
        Entropy_D.append(np.sum(np.square(cD_values[x]) * np.log(np.square(cD_values[x]))))
        Entropy_A.append(np.sum(np.square(cA_values[x]) * np.log(np.square(cA_values[x]))))
    return np.sum(cA_mean)/14,np.sum(cA_std)/14,np.sum(cD_mean)/14,np.sum(cD_std)/14,np.sum(cA_Energy)/14,np.sum(cD_Energy)/14,np.sum(Entropy_A)/14,np.sum(Entropy_D)/14

## Variance and Mean of Vertex to Vertex Slope

In [8]:
import heapq

from scipy.signal import argrelextrema

def first_diff(i):
    b=i
    
    
    out = np.zeros(len(b))
    
    for j in range(len(i)):
        out[j] = b[j-1]-b[j]# Obtaining the 1st Diffs
        
        j=j+1
        c=out[1:len(out)]
    return c

#first_diff(s)

def slope_mean(p):
    b = p #Extracting the data from the 14 channels
    output = np.zeros(len(b)) #Initializing the output array with zeros
    res = np.zeros(len(b)-1)
    
    k = 0; #For counting the current row no.
    for i in b:
        x=i
        amp_max = i[argrelextrema(x, np.greater)[0]]
        t_max = argrelextrema(x, np.greater)[0]
        amp_min = i[argrelextrema(x, np.less)[0]]
        t_min = argrelextrema(x, np.less)[0]
        t = np.concatenate((t_max,t_min),axis=0)
        t.sort()#sort on the basis of time

        h=0
        amp = np.zeros(len(t))
        res = np.zeros(len(t)-1)
        for l in range(len(t)):
            amp[l]=i[t[l]]
           
        
        amp_diff = first_diff(amp)
        
        t_diff = first_diff(t)
        
        for q in range(len(amp_diff)):
            res[q] = amp_diff[q]/t_diff[q]         
        output[k] = np.mean(res) 
        k=k+1
    return np.sum(output)/14




def first_diff(i):
    b=i
    
    
    out = np.zeros(len(b))
    
    for j in range(len(i)):
        out[j] = b[j-1]-b[j]# Obtaining the 1st Diffs
        
        j=j+1
        c=out[1:len(out)]
    return c #returns first diff


def slope_var(p):
    b = p #Extracting the data from the 14 channels
    output = np.zeros(len(b)) #Initializing the output array with zeros
    res = np.zeros(len(b)-1)
    
    k = 0; #For counting the current row no.
    for i in b:
        x=i
        amp_max = i[argrelextrema(x, np.greater)[0]]#storing maxima value
        t_max = argrelextrema(x, np.greater)[0]#storing time for maxima
        amp_min = i[argrelextrema(x, np.less)[0]]#storing minima value
        t_min = argrelextrema(x, np.less)[0]#storing time for minima value
        t = np.concatenate((t_max,t_min),axis=0) #making a single matrix of all matrix
        t.sort() #sorting according to time

        h=0
        amp = np.zeros(len(t))
        res = np.zeros(len(t)-1)
        for l in range(len(t)):
            amp[l]=i[t[l]]
           
        
        amp_diff = first_diff(amp)
        
        t_diff = first_diff(t)
        
        for q in range(len(amp_diff)):
            res[q] = amp_diff[q]/t_diff[q] #calculating slope        
    
        output[k] = np.var(res) 
        k=k+1#counting k
    return np.sum(output)/14

def wrapper3(epoch):
    var1 = slope_mean(epoch)
    var2 = slope_var(epoch)
    return var1,var2


## FFT features(Max Power)

In [30]:
from scipy import signal

def maxPwelch(data_win,Fs):
 
    
    BandF = [0.1, 3, 7, 12, 30]
    PMax = np.zeros([14,(len(BandF)-1)]);
    
    for j in range(14):
        f,Psd = signal.welch(data_win[j,:], Fs)
        
        for i in range(len(BandF)-1):
            fr = np.where((f>BandF[i]) & (f<=BandF[i+1]))
            PMax[j,i] = np.max(Psd[fr])
    
    return np.sum(PMax[:,0])/14,np.sum(PMax[:,1])/14,np.sum(PMax[:,2])/14,np.sum(PMax[:,3])/14


## Shanon Entropy and Entropy Spectral

In [10]:
def entropy(labels): # Shanon Entropy
    """ Computes entropy of 0-1 vector. """
    n_labels = len(labels)
    counts = np.bincount(labels)
    probs = counts[np.nonzero(counts)] / n_labels
    n_classes = len(probs)

    if n_classes <= 1:
        return 0
    return - np.sum(probs * np.log(probs)) / np.log(n_classes)



## Autoregressive model

### Autoregression model- Yule Walker Algorithm

In [11]:
def autogressiveModelParameters(labels):
    b_labels = len(labels)
    feature = []
    for i in range(14):
        coeff, sig = alg.AR_est_YW(labels[i,:], 11,)
        feature.append(coeff)
    a = []     
    for i in range(11):
        a.append(np.sum(feature[:][i])/14)
     
    return a

### Autoregression model- Burg Algorithm

In [12]:
def autogressiveModelParametersBurg(labels):
    feature = []
    feature1 = []
    model_order = 3
    for i in range(14):
        AR, rho, ref = arburg(labels[i], model_order)
        feature.append(AR);
    for j in range(14):
        for i in range(model_order):
            feature1.append(feature[j][i])

    return feature1

## Creating the feature vectors

In [39]:
lowfiles  = [f for f in listdir('Training-Data/real_EEG_data') if isfile(join('Training-Data/real_EEG_data', f))] 
#highfiles = [f for f in listdir('Training-Data/High') if isfile(join('Training-Data/High', f))]
files = []

for i in lowfiles:
    files.append([i,'real_EEG_data'])
    
#for i in highfiles:
 #   files.append([i,'High'])
    
mypath = 'Training-Data/'
csvfile = "Features/amlfeatures.csv"

with open(csvfile, "a") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerow(names) 
    for counter in range(len(files)):
        subfolder =  files[counter][1]
        tag = files[counter][1] 
        data_path = mypath + subfolder +'/'+files[counter][0]
        print(data_path)
        f = pyedflib.EdfReader(data_path)
        n = f.signals_in_file
        signal_labels = f.getSignalLabels()
        sigbufs = np.zeros((14, f.getNSamples()[3]))
        
        for i in np.arange(14):
            sigbufs[i,:] = f.readSignal(i)
        for i in np.arange(5,185,3):
            features = []
            epoch = sigbufs[:,i*128:(i+3)*128]
            #print(epoch)
            if len(epoch[0]) == 0:
                break
            
            # Hjorth Parameters
            feature_list = hjorth(epoch)
            for feat in feature_list:
                features.append(feat)
        
            #Kurtosis , 2nd Diff Mean, 2nd Diff Max
            feature_list = wrapper1(epoch)
            for feat in feature_list:
                features.append(feat)
            
            #Coeffeicient of Variation
            feat = coeff_var(epoch)
            features.append(feat)
            
            #Skewness , 1st Difference Mean, 1st Difference Max
            feature_list = wrapper2(epoch)
            for feat in feature_list:
                features.append(feat)
            
            
            # wavelet transform features 
            feature_list = wavelet_features(epoch)
            for feat in feature_list:
                features.append(feat)
        
        
            # Variance and mean of Vertex to Vertex Slope
            feature_list = wrapper3(epoch)
            for feat in feature_list:
                features.append(feat)
            
            
            #Fast Fourier Transform features(Max Power)
            feature_list  =  maxPwelch(epoch,128)
            for feat in feature_list:
                features.append(feat)
        
            #Autoregressive model Coefficients
            feature_list = autogressiveModelParametersBurg(epoch)
            for feat in feature_list:
                features.append(feat.real)
            
            features.append(tag);
        
            writer.writerow(features) 

Training-Data/real_EEG_data/1.edf
[[  43.1051957    39.02517739   44.12520027 ... -151.20567636
  -147.12565805 -155.28569467]
 [  62.48528267   63.50528725   63.50528725 ...   83.39537652
    82.37537194   82.37537194]
 [-277.1762417  -286.3562829  -276.15623713 ...  190.49585718
   188.45584802  181.31581598]
 ...
 [ -40.53517967  -27.78512245    7.91503777 ...  -89.4953994
   -89.4953994   -99.69544518]
 [  16.07507439   17.09507897   18.62508583 ... -693.8481117
  -688.74808881 -676.50803388]
 [  66.56530098   53.30524147   57.38525978 ... -127.23556878
  -130.29558251 -135.90560769]]
[[-162.42572671 -160.38571756 -169.05575647 ... -140.49562829
  -140.49562829 -140.49562829]
 [  79.31535821   67.58530556   59.42526894 ...   85.43538567
    87.47539483   87.47539483]
 [ 172.13577478  164.48574044  160.40572213 ... -168.03575189
  -165.99574273 -173.13577478]
 ...
 [-111.93550011 -101.73545434  -96.63543145 ...   68.60531014
    77.78535134   93.59542229]
 [-654.57793545 -639.787869

[[ -65.52529183  -63.48528267  -70.11531243 ...  133.37560082
   129.29558251  123.17555505]
 [  36.47516594   29.3351339    21.68509957 ...   37.49517052
    36.47516594   34.43515679]
 [-162.42572671 -161.40572213 -164.97573816 ... -218.0159762
  -223.11599908 -228.21602197]
 ...
 [  64.52529183   86.96539254   70.6453193  ...  166.01574731
   173.66578164  150.71567864]
 [ 134.90560769  129.8055848   115.01551843 ...   68.09530785
    70.6453193    68.60531014]
 [ -88.98539712  -80.82536049  -87.96539254 ...  121.64554818
   111.4455024   111.4455024 ]]
[[ 126.23556878  129.29558251  125.2155642  ...    7.40503548
     7.40503548   10.9750515 ]
 [  34.43515679   36.47516594   35.45516136 ...   12.50505837
    21.68509957   24.7451133 ]
 [-226.17601282 -226.17601282 -228.21602197 ... -225.15600824
  -228.21602197 -225.15600824]
 ...
 [ 145.61565576  153.77569238  134.90560769 ...  100.73545434
    80.84536507   75.74534218]
 [  73.70533303   74.72533761   61.97528038 ...  103.7954680

[[ -23.19510185  -21.1550927   -21.1550927  ...   85.94538796
    77.27534905   95.12542916]
 [  36.47516594   36.47516594   34.43515679 ...   11.48505379
     9.44504463   10.46504921]
 [-127.23556878 -130.29558251 -130.29558251 ... -127.23556878
  -128.25557336 -130.29558251]
 ...
 [  96.65543603  100.73545434   64.01528954 ...   51.77523461
    57.89526207   36.98516823]
 [ 121.64554818  116.035523    108.89549096 ...   36.98516823
    28.82513161   35.96516365]
 [ -46.14520485  -48.18521401  -48.18521401 ...   42.59519341
    36.47516594   40.55518425]]
[[  96.14543374   80.33536278   78.29535363 ...  -21.1550927
   -18.09507897  -27.27512016]
 [  13.52506294   23.72510872   22.70510414 ...   24.7451133
    26.78512245   17.60508125]
 [-131.31558709 -130.29558251 -129.27557794 ... -158.3457084
  -160.38571756 -161.40572213]
 ...
 [  21.68509957   48.71522087   68.60531014 ...   71.66532387
    84.92538338   43.61519799]
 [  49.73522545   50.75523003   47.6952163  ...  113.99551385


[[ -47.16520943  -58.38525978  -54.30524147 ...   35.96516365
    39.02517739   35.96516365]
 [  38.5151751    28.31512932   23.72510872 ...   12.50505837
    14.54506752   20.66509499]
 [ -97.14543374 -104.28546578 -100.20544747 ... -110.40549325
  -108.36548409 -106.32547494]
 ...
 [ -59.91526665  -95.61542687 -101.73545434 ...  117.56552987
   110.93550011   83.90537881]
 [  -2.7950103   -14.52506294  -20.64509041 ...   64.01528954
    44.63520256   58.91526665]
 [ -78.27534905  -90.00540169  -97.14543374 ...  -12.99505608
   -12.99505608  -14.01506065]]
[[  34.94515908   36.98516823   40.04518196 ...  -23.19510185
   -23.19510185  -23.19510185]
 [  13.52506294    4.34502174   15.5650721  ...    7.40503548
     3.32501717    1.28500801]
 [-105.30547036 -103.2654612  -101.22545205 ...  -98.16543832
   -98.16543832 -103.2654612 ]
 ...
 [  89.00540169   89.00540169   93.59542229 ...  -71.6453193
   -82.86536965  -83.88537423]
 [  81.86536965   73.70533303   70.6453193  ...   76.7653467

[[ -269.01620508  -266.97619593  -266.97619593 ...  -615.30775921
   -638.25786221  -636.21785306]
 [  115.01551843   119.09553674   126.74557107 ...  1839.84325933
   1854.12332341  1836.78324559]
 [ -295.5363241   -286.3562829   -304.7163653  ... -3240.79954223
  -3279.55971618 -3254.56960403]
 ...
 [  376.64669261   377.66669718   355.73659876 ...  -589.80764477
   -637.23785763  -645.90789654]
 [  157.85571069   160.91572442   172.13577478 ... -1220.17047379
  -1264.54067292 -1281.37074846]
 [ -398.04678416  -403.14680705  -405.18681621 ... -1254.34062715
  -1270.1506981  -1263.52066835]]
[[ -629.07782101  -628.05781643  -616.83776608 ...  -487.29718471
   -478.11714351  -480.15715267]
 [ 1825.05319295  1820.97317464  1817.91316091 ...   -58.38525978
    -55.83524834   -46.65520714]
 [-3229.57949187 -3219.3794461  -3196.4293431  ...    71.15532158
     84.92538338    86.96539254]
 ...
 [ -652.02792401  -672.93801785  -678.03804074 ...   145.61565576
    182.33582055   178.25580224]

...

[[-525.03735409 -515.85731289 -484.23717098 ... -201.18590066
  -201.18590066 -182.82581827]
 [  15.5650721    16.58507668   11.48505379 ...    8.42504005
     7.40503548   12.50505837]
 [  49.22522316   27.29512474    7.91503777 ...  -25.23511101
    -5.34502174   -7.3850309 ]
 ...
 [-110.91549554 -110.40549325 -119.07553216 ...   41.57518883
    36.47516594   26.27512016]
 [ -49.20521859  -40.02517739  -46.14520485 ...  -51.24522774
   -47.16520943  -35.94515908]
 [ -53.79523919  -68.07530327  -71.6453193  ...  -87.96539254
   -73.68532845  -75.72533761]]
[[-200.16589609 -208.835935   -192.00585946 ...  -41.04518196
   -50.22522316  -68.07530327]
 [  12.50505837    4.34502174    4.34502174 ...    9.44504463
    14.54506752   12.50505837]
 [ -33.39514763  -17.58507668   10.9750515  ...  -44.1051957
   -39.5151751   -40.53517967]
 ...
 [  26.27512016   26.27512016   27.29512474 ...   42.08519112
    40.55518425   38.5151751 ]
 [ -48.18521401  -57.36525521  -55.32524605 ...   12.5050583

[[-863.1688716  -862.14886702 -845.82879377 ... -357.75660334
  -340.92652781 -336.8465095 ]
 [   1.28500801   -5.85502403   -6.87502861 ...   10.46504921
    16.58507668    5.36502632]
 [-518.40732433 -519.42732891 -508.20727855 ...  -40.53517967
   -25.23511101  -41.55518425]
 ...
 [  24.23511101   21.17509728   21.17509728 ...  126.23556878
   138.47562371  131.33559167]
 [ 184.37582971  165.50574502  159.38571756 ...  -54.30524147
   -48.18521401  -55.32524605]
 [-697.92813001 -713.73820096 -710.67818723 ... -136.92561227
  -117.035523   -128.76557565]]
[[-350.10656901 -371.01666285 -361.83662165 ... -318.99642939
  -331.74648661 -364.89663539]
 [   2.30501259    8.42504005    8.42504005 ...   16.58507668
    16.58507668   10.46504921]
 [ -63.48528267  -87.45539025  -88.47539483 ...  -76.23533989
   -76.23533989  -77.25534447]
 ...
 [ 128.27557794  136.43561456  138.47562371 ...  -15.03506523
   -19.11508354  -30.3351339 ]
 [ -54.30524147  -51.24522774  -51.24522774 ... -106.325474

Training-Data/real_EEG_data/15.edf
[[ -246.06610208  -246.06610208  -249.12611582 ...  -117.54552529
   -142.53563745  -153.24568551]
 [ 8564.73344015  8564.73344015  8564.73344015 ...  8564.73344015
   8564.73344015  8564.73344015]
 [ -248.61611353  -250.65612268  -247.59610895 ...  -162.42572671
   -177.72579538  -189.45584802]
 ...
 [-7263.18759442 -7263.18759442 -7263.18759442 ... -3943.07269398
  -3945.11270314 -3956.33275349]
 [  788.21853971   782.09851225   783.11851682 ...   -15.03506523
    -14.01506065   -19.11508354]
 [ -147.12565805  -160.38571756  -163.44573129 ...  -127.23556878
   -140.49562829  -137.43561456]]


C:\Users\mirab\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


ValueError: negative dimensions are not allowed

## Normalization( Wait till the csv file gets populated then run the code below after the features are extracted)

In [40]:
r = csv.reader(open('Features/amlfeatures.csv')) # Here your csv file
lines = [l for l in r]

In [42]:
for i in range(len(lines[1])-1):  
    columns = []
    for j in range(1,len(lines)):
        columns.append(float(lines[j][i]))
    mean = np.mean(columns,axis = 0)
    std_dev  = np.std(columns,axis = 0)
    
    for j in range(1,len(lines)):
        lines[j][i] = (float(lines[j][i])-mean)/std_dev

writer = csv.writer(open('Features/Normalizedamlfeatures.csv', 'w')) # This file will store the normalized features
writer.writerows(lines)


In [27]:
f.__del__()